In [99]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Lambda, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras import backend as K

In [100]:
# Define the sentence and tokenize
sentence = "The quick brown fox jumps over the lazy dog"
words = sentence.lower().split()

# Tokenize the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([sentence])
word2id = tokenizer.word_index

id2word = {v: k for k, v in word2id.items()}


vocab_size = len(word2id) + 1  # Adding 1 because index 0 is reserved in Keras


In [101]:
print(id2word)

{1: 'the', 2: 'quick', 3: 'brown', 4: 'fox', 5: 'jumps', 6: 'over', 7: 'lazy', 8: 'dog'}


In [102]:
# Convert sentence to sequence of word IDs
word_ids = [word2id[word] for word in words]

In [103]:
word_ids

[1, 2, 3, 4, 5, 6, 1, 7, 8]

In [104]:
# Parameters
context_window = 2  # Use 2 words before and 2 words after as context


In [105]:
# Generate CBOW pairs (context words -> target word)
def generate_context_target_pairs(word_ids, window_size):
    pairs = []
    for i in range(window_size, len(word_ids) - window_size):
        context = word_ids[i - window_size:i] + word_ids[i + 1:i + window_size + 1]
        target = word_ids[i]
        pairs.append((context, target))
    return pairs

In [106]:
pairs = generate_context_target_pairs(word_ids, context_window)

In [107]:
pairs

[([1, 2, 4, 5], 3),
 ([2, 3, 5, 6], 4),
 ([3, 4, 6, 1], 5),
 ([4, 5, 1, 7], 6),
 ([5, 6, 7, 8], 1)]

In [108]:
# Separate context and target from pairs
contexts, targets = zip(*pairs)

In [109]:
print(targets, contexts)

(3, 4, 5, 6, 1) ([1, 2, 4, 5], [2, 3, 5, 6], [3, 4, 6, 1], [4, 5, 1, 7], [5, 6, 7, 8])


In [110]:
# Convert to NumPy arrays
contexts = np.array(contexts)
targets = np.array(targets)

In [111]:
# Dimension of the embedding space
embedding_dim = 50  

# Define inputs for the context words
# Context has 4 words (2 before, 2 after)
context_input = Input(shape=(context_window * 2,), dtype='int32')  
# Embedding layer
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=context_window * 2)
context_embeddings = embedding(context_input)

# Average the context embeddings
average_embedding = Lambda(lambda x: K.mean(x, axis=1))(context_embeddings)

# Dense output layer to predict the target word
output = Dense(vocab_size, activation='softmax')(average_embedding)

# Define and compile the model
model = Model(inputs=context_input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_6 (Embedding)         │ (None, 4, 50)          │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_6 (Lambda)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 9)              │           459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 909 (3.55 KB)

 Trainable params: 909 (3.55 KB)

 Non-trainable params: 0 (0.00 B)

None


In [112]:
# Train the model
model.fit(contexts, targets, epochs=100, batch_size=4, verbose=1)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 2.2062  
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2167 - loss: 2.1922
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4333 - loss: 2.1890
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7833 - loss: 2.1823
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7833 - loss: 2.1745
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7833 - loss: 2.1681
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7833 - loss: 2.1619
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7833 - loss: 2.1539
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8667 - loss: 2.1419
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8667 - loss: 2.1347
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8667 - loss: 2.1278
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8667 - loss: 2.

In [113]:
# Example context: ["quick", "brown", "jumps", "over"]
test_context = ["quick", "brown", "jumps", "over"]
test_context_ids = np.array([word2id[word] for word in test_context]).reshape(1, -1)

test_context_ids.shape

(1, 4)

In [114]:
# Predict the target word
predicted = model.predict(test_context_ids)

predicted

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


array([[0.05836796, 0.12776595, 0.04455542, 0.13296542, 0.32241145,
        0.1403813 , 0.08811285, 0.04363702, 0.04180264]], dtype=float32)

In [115]:
predicted_word_id = np.argmax(predicted)
predicted_word = id2word[predicted_word_id]

print("Predicted word:", predicted_word)

Predicted word: fox
